In [8]:
import sys
import warnings
import numpy as np
import time
import pandas as pd
import atexit
from time import time, strftime, localtime
from datetime import timedelta
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler



np.set_printoptions(suppress=True)
np.warnings.filterwarnings('ignore')
np.set_printoptions(threshold=sys.maxsize)



def sigmoid(x):

    return 1 / (1 + np.exp(-x))

def sigmoid_approximate_poly_7(x):
    
    return 0.5 + 1.73496 * (x/8) - 4.19407 * (x/8)**3 + 5.43402 * (x/8)** 5 - 2.50739 *(x/8) ** 7

def sigmoid_approximate_poly_5(x):
    
    return  0.5 + (1.53048 * (x/8)) - (2.3533056 * ((x/8) ** 3)) + (1.3511295 * ((x/8) ** 5))


def sigmoid_approximate_poly_3(x):

    return 0.5 + 1.20096 * (x/8) - 0.81562 * (x/8)**3

def sigmoid_approximate(x):

    return (1 / 2) + (x / 4) - (x**3 / 48) + (x**5 / 480)


def share(input):
    np.random.seed(2)
    x = np.random.random((np.shape(input)))
    y = 0.1 *np.random.random((np.shape(input)))
    z = input - x - y

    return x , y , z

tol=1e-6

max_iter = 100



###################################################################################
# #Reading dataset
# data = pd.read_csv('lbw.csv')

# #Separation of Y and X from Dataset 
# np.random.seed(1)

# Y1 = data.iloc[:,-1:]
# y = Y1.values.reshape(np.shape(data)[0],1)
# data.drop(data.columns[len(data.columns)-1], axis=1, inplace=True)
# X = np.array(data)


# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state =0)


###################################################################################
#NM = X_train.T.dot(X_train)
#print(-4 * np.linalg.inv(NM))

# print(y_train)


###################################################################################

# np.random.seed(1)
# df = pd.read_csv("cancerdata.csv")
# X = df.values[:,2:-1].astype('float64')
# # print('\n ###################################### \n',X)

# X = (X - np.mean(X, axis =0)) /  np.std(X, axis = 0)
# # print('\n ###################################### \n',X)

# ## Add a bias column to the data
# X = np.hstack([np.ones((X.shape[0], 1)),X])
# # print('\n ###################################### \n',X)

# X = MinMaxScaler().fit_transform(X)
# # print('\n ###################################### \n',X)

# np.random.seed(1)
# Y = df["diagnosis"].map({'M':1,'B':0})
# Y = np.array(Y)

# X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.25)

###################################################################################

# from sklearn.datasets import load_breast_cancer
# breast_cancer = load_breast_cancer()
# X = breast_cancer.data
# z= breast_cancer.target
# y = z.reshape(569,1)
# # np.random.seed(2)
# # z = np.random.randint(2, size=569)
# # y = z.reshape(569,1)


# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)

###################################################################################

# #Reading dataset
# np.random.seed(2)
# data = pd.read_csv('nhanes3.csv')

# print(data.shape)

# # Separation of Y and X from Dataset
# Y = np.array(data['Outcome'])
# y = Y.reshape(768, 1)
# del data['Outcome']
# X = np.array(data)
# np.random.seed(2)

# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state =0)

# print(X_train)
# print(y_train)


###################################################################################

# sizex = 100
# sizey = 20

# rs12 = np.random.RandomState(1)
# X_train = np.round(rs12.uniform(-1, 1, (sizex, sizey)), 7)


# rs13 = np.random.RandomState(2)
# y_train = rs13.randint(0, 2, (sizex, 1))




###################################################################################
#Reading dataset
data = pd.read_csv('lbw.csv')

#Separation of Y and X from Dataset 
np.random.seed(1)

Y1 = data.iloc[:,-1:]
y = Y1.values.reshape(np.shape(data)[0],1)
data.drop(data.columns[len(data.columns)-1], axis=1, inplace=True)
X = np.array(data)


X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state =0)

###################################################################################

def secondsToStr(elapsed=None):
    if elapsed is None:
        return strftime("%Y-%m-%d %H:%M:%S", localtime())
    else:
        return str(timedelta(seconds=elapsed))

def log(s, elapsed=None):
    line = "="*40
    print(line)
    print(secondsToStr(), '-', s)
    if elapsed:
        print("Elapsed time:", elapsed)
    print(line)
    print()

def endlog():
    end = time()
    elapsed = end-start
    log("End Program", secondsToStr(elapsed))
    return elapsed

def newton_step(curr, X):
    
#     print(np.max(X.dot(curr[:, 0])))
#     print(np.min(X.dot(curr[:, 0])))
                  
#     print((X.dot(curr[:, 0])).T)
#     print((X.dot(curr[:, 0])))

   
    p = np.array(sigmoid(X.dot(curr[:, 0])), ndmin=2).T
    
#     print(p)
    
    W = np.diag((p * (1 - p))[:, 0])

    hessian = X.T.dot(W).dot(X)
    
    
    grad = X.T.dot(y_train - p)

#     print(hessian)
    
    step = np.dot(np.linalg.pinv(hessian), grad)
  

    beta = curr + step

    return beta

def check_coefs_convergence(beta_old, beta_new, tol, iters):

    coef_change = np.abs(beta_old - beta_new)
    
#     print('\n coef_change: ' , np.max(coef_change), '\n')

    return not (np.any(coef_change > tol) & (iters < max_iter))

start = time()
# atexit.register(endlog)
# log("Start Program")

beta_old, beta = np.zeros((np.size(X_train, 1), 1)), np.zeros((np.size(X_train, 1), 1))
# beta_old, beta = beta_old, beta +   1
iter_count = 0

coefs_converged = False

while not coefs_converged:

    beta_old = beta

    beta = newton_step(beta, X_train)
    
    iter_count += 1

    coefs_converged = check_coefs_convergence(beta_old, beta, tol, iter_count)

end = time()

elapsed = end - start
print('Elapsed :', end - start)
print('Iterations : {}'.format(iter_count))
print('Beta : \n\n{}'.format(beta))
# endlog()

###################################################################################
# from sklearn.metrics import accuracy_score
# predict = sigmoid(X_test.dot(beta))
# prediction = ((predict) > 0.5)
# y_predicted = prediction.astype(int)



# print(y_test.T, '\n')

# print(y_predicted.T, '\n')

# print('accuracy_score: ', accuracy_score(y_test, y_predicted))

# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import recall_score
# from sklearn.metrics import precision_score
# from sklearn.metrics import f1_score

# print('confusion_matrix: \n\n' , confusion_matrix(y_test, y_predicted), '\n')
# print('accuracy_score: \n\n', accuracy_score(y_test, y_predicted), '\n')
# print('recall_score: \n\n', recall_score(y_test, y_predicted), '\n')
# print('precision_score: \n\n', precision_score(y_test, y_predicted), '\n')
# print('f1_score: \n\n', f1_score(y_test, y_predicted), '\n')



Elapsed : 0.0030298233032226562
Iterations : 6
Beta : 

[[ 0.01574389]
 [ 0.01127234]
 [ 0.78666461]
 [-0.47132401]
 [-1.32410422]
 [-0.75584258]
 [-2.20748099]
 [-0.96060737]
 [-0.24569154]]


In [2]:
[[ -0.00006262]
 [  0.08016732]
 [  1.11380032]
 [  3.31025933]
 [  2.8345651 ]
 [-20.18454387]
 [ -0.0050867 ]
 [ -0.00857207]
 [  0.17974795]
 [-22.2778775 ]
 [ -0.81552672]
 [-22.22518061]
 [ -1.8374335 ]
 [ -0.81552672]
 [  0.00251954]]


%matplotlib inline

import sys
import math
import numpy as np
import warnings
import time
import atexit
from patsy import dmatrices
from numpy.linalg import inv
from time import time, strftime, localtime
from datetime import timedelta
import matplotlib.pyplot as plt

np.set_printoptions(suppress=True)
np.warnings.filterwarnings('ignore')
np.set_printoptions(threshold=sys.maxsize)
np.random.seed(1) # set the seed

def fullprint(*args, **kwargs):
  from pprint import pprint
  import numpy
  opt = numpy.get_printoptions()
  numpy.set_printoptions(threshold=numpy.inf)
  pprint(*args, **kwargs)
  numpy.set_printoptions(**opt)



#Reading dataset
np.random.seed(2)
data = pd.read_csv('nhanes3.csv'')

# Separation of Y and X from Dataset
Y = np.array(data['Outcome'])
y = Y.reshape(768, 1)
del data['Outcome']
X = np.array(data)
np.random.seed(2)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state =0)


n = 10

beta_old, beta = np.zeros((np.size(X_train,1),1)), np.zeros((np.size(X_train,1),1))


def sigmoid(x):

    return 1 / (1 + np.exp(-x))

def sigmoid_approximate_poly_7(x):
    
    return 0.5 + 1.73496 * (x/8) - 4.19407 * (x/8)**3 + 5.43402 * (x/8)** 5 - 2.50739 *(x/8) ** 7

def sigmoid_approximate_poly_5(x):
    
    return  0.5 + (1.53048 * (x/8)) - (2.3533056 * ((x/8) ** 3)) + (1.3511295 * ((x/8) ** 5))


def sigmoid_approximate_poly_3(x):

    return 0.5 + 1.20096 * (x/8) - 0.81562 * (x/8)**3

def sigmoid_approximate(x):

    return (1 / 2) + (x / 4) - (x**3 / 48) + (x**5 / 480)


hessian_temp = (X_train.T.dot(X_train))

# print(hessian_temp)

hessian = -4 * inv(hessian_temp)


# print(hessian)

for i in range(0, n):
    
    beta_old = beta
                

    p = (np.array(sigmoid(X_train.dot(beta_old[:,0])), ndmin=2)).T
    
#     print(X_train)
#     print(beta_old)
#     print((X_train.dot(beta_old[:,0]*1/8)).T)

    grad = X_train.T.dot(y_train-p)
    
#     print(p)

    step= hessian.dot(grad)
    
#     fullprint(step)

    beta = beta_old - step


    
print('Beta : \n\n{}'.format(beta))
# fullprint(beta)

SyntaxError: EOL while scanning string literal (<ipython-input-2-d83ad4601c4d>, line 49)

In [ ]:
rs14 = np.random.RandomState(2)
j = rs14.uniform(0, 1, (2,2))

print(j)

In [ ]:
# ccc = -1/4 * X_train.T.dot(X_train)

# d = np.linalg.inv(ccc)
# print(d)

In [ ]:
# from sklearn.metrics import accuracy_score
# beta = np.array([[ 0.17142328],
#  [ 0.01402442],
#  [-0.03110796],
#  [ 0.00077171],
#  [ 0.00073807],
#  [-0.00326994],
#  [ 0.41085089],
#  [-0.02538718]])

# predict = sigmoid(X_test.dot(beta))
# prediction = ((predict) > 0.5)
# y_predicted = prediction.astype(int)


# from sklearn.metrics import accuracy_score
# beta1 = np.array([[ 0.18608374],
#  [ 0.01509076],
#  [-0.0336688 ],
#  [ 0.00080547],
#  [ 0.00083535],
#  [-0.0037617 ],
#  [ 0.44514595],
#  [-0.02737736]])

# predict1 = sigmoid(X_test.dot(beta1))
# prediction1 = ((predict1) > 0.5)
# y_predicted1 = prediction1.astype(int)
    

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

print('confusion_matrix: \n\n' , confusion_matrix(y_test, y_predicted), '\n')
print('accuracy_score: \n\n', accuracy_score(y_test, y_predicted), '\n')
print('recall_score: \n\n', recall_score(y_test, y_predicted), '\n')
print('precision_score: \n\n', precision_score(y_test, y_predicted), '\n')
print('f1_score: \n\n', f1_score(y_test, y_predicted), '\n')

In [ ]:
from sklearn.metrics import roc_curve

fpr_RF, tpr_RF, thresholds_RF = roc_curve(y_test, predict)
# fpr_LR, tpr_LR, thresholds_LR = roc_curve(y_test, predict1)
# print(predict)

import matplotlib.pyplot as plt

plt.plot(fpr_RF, tpr_RF,'r-',label = 'LR_SIG')
# plt.plot(fpr_LR,tpr_LR,'b-', label= 'LR_7')
plt.plot([0,1],[0,1],'k-',label='random')
plt.plot([0,0,1,1],[0,1,1,1],'g-',label='perfect')
plt.legend()
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score

auc_RF = roc_auc_score(y_test, predict)
# auc_LR = roc_auc_score(y_test, predict1)

print('AUC RF:%.3f'% auc_RF)
# print('AUC LR:%.3f'% auc_LR)

import matplotlib.pyplot as plt
plt.plot(fpr_RF, tpr_RF,'y-',label = 'SIG AUC: %.3f'%auc_RF)
# plt.plot(fpr_LR,tpr_LR,'b-', label= '7 AUC: %.3f'%auc_LR)
plt.plot([0,1],[0,1],'k-',label='random')
plt.plot([0,0,1,1],[0,1,1,1],'g-',label='perfect')
plt.legend()
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve

fpr_RF, tpr_RF, thresholds_RF = roc_curve(y_test, predict)
fpr_LR, tpr_LR, thresholds_LR = roc_curve(y_test, predict1)
# print(predict)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(fpr_RF, tpr_RF,'r-',label = 'LR_SIG')
plt.plot(fpr_LR,tpr_LR,'b-', label= 'LR_7')
plt.plot([0,1],[0,1],'k-',label='random')
plt.plot([0,0,1,1],[0,1,1,1],'g-',label='perfect')
plt.legend()
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score

auc_RF = roc_auc_score(y_test, predict)
auc_LR = roc_auc_score(y_test, predict1)

print('AUC RF:%.3f'% auc_RF)
print('AUC LR:%.3f'% auc_LR)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(fpr_RF, tpr_RF,'y-',label = 'SIG AUC: %.3f'%auc_RF)
plt.plot(fpr_LR,tpr_LR,'b-', label= '7 AUC: %.3f'%auc_LR)
plt.plot([0,1],[0,1],'k-',label='random')
plt.plot([0,0,1,1],[0,1,1,1],'g-',label='perfect')
plt.legend()
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

In [ ]:
# from sklearn.datasets import load_diabetes
# load_diabetes = load_diabetes()
# X = diabetes.data
# print(np.amax(X))

In [ ]:
import sys
import warnings
import numpy as np
import time
import pandas as pd
import atexit
from time import time, strftime, localtime
from datetime import timedelta
import matplotlib.pyplot as plt

np.set_printoptions(suppress=True)
np.warnings.filterwarnings('ignore')
np.set_printoptions(threshold=sys.maxsize)
np.random.seed(1) # set the seed

def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)


def sigmoid(x):


    return 1 / (1 + np.exp(-x))

def sigmoid_approximate(x):

    return (1 / 2) + (x / 4) - (x**3 / 48) + (x**5 / 480)


def share(input):
    np.random.seed(2)
    x = np.random.random((np.shape(input)))
    y = 0.1 *np.random.random((np.shape(input)))
    z = input - x - y

    return x , y , z

tol=1e-6

max_iter = 1000


# #Reading dataset
# data = pd.read_csv('diabetes.csv')

# #Separation of Y and X from Dataset 
# Y = np.array(data['Outcome'])
# y = Y.reshape(768,1)
# #print(y.shape)
# del data['Outcome']
# X = np.array(data)
# #print(X)



# from sklearn.datasets import load_breast_cancer
# breast_cancer = load_breast_cancer()
# X = breast_cancer.data
# z= breast_cancer.target
# y = z.reshape(569,1)
# np.random.seed(2)
# z = np.random.randint(2, size=569)
# y = z.reshape(569,1)


# sizex = 10000
# sizey = 100

# rs12 = np.random.RandomState(12)
# XA = np.round(rs12.uniform(-1,1,(sizex,sizey)),7)
# #XA1 , XA2 , XA3 = share(XA)


# rs13 = np.random.RandomState(13)
# YA = rs13.randint(0 , 2 , (sizex,1))
# #YA1 , YA2 , YA3 = sharey(YA)

# rs14 = np.random.RandomState(14)
# XB = np.round(rs14.uniform(-1,1,(sizex,sizey)),7)
# #XB1 , XB2 , XB3 = share(XB)

# rs15 = np.random.RandomState(15)
# YB = rs15.randint(0 , 2 , (sizex,1))
# #YB1 , YB2 , YB3 = sharey(YB)



###################################################################################

# sizex = 1000
# sizey = 500

# np.random.seed(4)
# XA = np.round(np.random.uniform(-40,40,(sizex,sizey)),7)
# #XA1 , XA2 , XA3 = share(XA)


# np.random.seed(3)
# YA = np.random.randint(0 , 2 , (sizex,1))
# #YA1 , YA2 , YA3 = sharey(YA)

# np.random.seed(5)
# XB = np.round(np.random.uniform(-40,40,(sizex,sizey)),7)
# #XB1 , XB2 , XB3 = share(XB)

# np.random.seed(7)
# YB = np.random.randint(0 , 2 , (sizex,1))
# #YB1 , YB2 , YB3 = sharey(YB)



###################################################################################
sizex = 6
sizey = 3

rs12 = np.random.RandomState(11)
XA = np.round(rs12.uniform(-1, 1, (sizex, sizey)), 7)
# XA1, XA2 = share(XA, 4)

rs13 = np.random.RandomState(22)
YA = rs13.randint(0, 2, (sizex, 1))
# YA1, YA2 = sharey(YA)

rs14 = np.random.RandomState(33)
XB = np.round(rs14.uniform(-1, 1, (sizex, sizey)), 7)
# XB1, XB2 = share(XB, 6)

rs15 = np.random.RandomState(44)
YB = rs15.randint(0, 2, (sizex, 1))
# YB1, YB2 = sharey(YB))

X = np.concatenate((XA,XB) , axis=0)
y = np.concatenate((YA,YB) , axis=0)

###################################################################################


# np.random.seed(3)
# XA = np.random.random((90, 10))
# XA1, XA2, XA3 = share(XA)

# np.random.seed(2)
# YA = np.random.randint(0, 2, (90, 1))
# YA1, YA2, YA3 = share(YA)

# np.random.seed(2)
# XB = np.random.random((90, 10))

# XB1, XB2, XB3 = share(XB)

# np.random.seed(2)
# YB = np.random.randint(0, 2, (90, 1))
# YB1, YB2, YB3 = share(YB)
###################################################################################

# np.random.seed(3)
# XA = np.random.random((100, 10))
# XA1, XA2, XA3 = share(XA)

# np.random.seed(2)
# YA = np.random.randint(0, 2, (100, 1))
# YA1, YA2, YA3 = share(YA)

# np.random.seed(2)
# XB = np.random.random((100, 10))

# XB1, XB2, XB3 = share(XB)

# np.random.seed(2)
# YB = np.random.randint(0, 2, (100, 1))
# YB1, YB2, YB3 = share(YB)

###################################################################################

def secondsToStr(elapsed=None):
    if elapsed is None:
        return strftime("%Y-%m-%d %H:%M:%S", localtime())
    else:
        return str(timedelta(seconds=elapsed))

def log(s, elapsed=None):
    line = "="*40
    print(line)
    print(secondsToStr(), '-', s)
    if elapsed:
        print("Elapsed time:", elapsed)
    print(line)
    print()

def endlog():
    end = time()
    elapsed = end-start
    log("End Program", secondsToStr(elapsed))
    return elapsed

def newton_step(curr, X):

    p = np.array(sigmoid(X.dot(curr[:, 0])), ndmin=2).T
    
    W = np.diag((p * (1 - p))[:, 0])

    hessian = X.T.dot(W).dot(X)
    
    grad = X.T.dot(y - p)

    step = np.dot(np.linalg.inv(hessian), grad)
  
    beta = curr + step

    return beta

def check_coefs_convergence(beta_old, beta_new, tol, iters):

    coef_change = np.abs(beta_old - beta_new)

    return not (np.any(coef_change > tol) & (iters < max_iter))

start = time()
atexit.register(endlog)
log("Start Program")

beta_old, beta = np.zeros((np.size(X, 1), 1)), np.zeros((np.size(X, 1), 1))

iter_count = 0

coefs_converged = False

while not coefs_converged:

    beta_old = beta
    #print(beta)
#    start_time = time.time()

    beta = newton_step(beta, X)
    
    iter_count += 1

    #print('Beta : \n\n{}'.format(beta))

    coefs_converged = check_coefs_convergence(beta_old, beta, tol, iter_count)

    
print('Iterations : {}'.format(iter_count))
print('Beta : \n\n{}'.format(beta))
endlog()

In [ ]:
import sys
import warnings
import numpy as np
import time
import pandas as pd
import atexit
from time import time, strftime, localtime
from datetime import timedelta
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler



np.set_printoptions(suppress=True)
np.warnings.filterwarnings('ignore')
np.set_printoptions(threshold=sys.maxsize)



def sigmoid(x):

    return 1 / (1 + np.exp(-x))

def sigmoid_approximate_poly_7(x):
    
    return 0.5 + 1.73496 * (x/8) - 4.19407 * (x/8)**3 + 5.43402 * (x/8)** 5 - 2.50739 *(x/8) ** 7


def sigmoid_approximate_poly_3(x):

    return 0.5 + 1.20096 * (x/8) - 0.81562 * (x/8)**3

def sigmoid_approximate(x):

    return (1 / 2) + (x / 4) - (x**3 / 48) + (x**5 / 480)


def share(input):
    np.random.seed(2)
    x = np.random.random((np.shape(input)))
    y = 0.1 *np.random.random((np.shape(input)))
    z = input - x - y

    return x , y , z

tol=1e-6

max_iter = 1


# #Reading dataset
# data = pd.read_csv('diabetes_w.csv')

# #Separation of Y and X from Dataset 
# np.random.seed(1)

# Y1 = data.iloc[:,-1:]
# y = Y1.values.reshape(np.shape(data)[0],1)

# # last_colomn = data.iloc[:,-1:]
# # print('last_colomn: ' , data)
# # print(y.shape)
# #print(y.shape)
# # print(data.iloc[-1:,-1:])
# # del data['last_colomn']
# data.drop(data.columns[len(data.columns)-1], axis=1, inplace=True)
# X = np.array(data)
# #print(X)


# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)


###################################################################################

# np.random.seed(1)
# df = pd.read_csv("cancerdata.csv")
# X = df.values[:,2:-1].astype('float64')
# # print('\n ###################################### \n',X)

# X = (X - np.mean(X, axis =0)) /  np.std(X, axis = 0)
# # print('\n ###################################### \n',X)

# ## Add a bias column to the data
# X = np.hstack([np.ones((X.shape[0], 1)),X])
# # print('\n ###################################### \n',X)

# X = MinMaxScaler().fit_transform(X)
# # print('\n ###################################### \n',X)

# np.random.seed(1)
# Y = df["diagnosis"].map({'M':1,'B':0})
# Y = np.array(Y)

# X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.25)

###################################################################################

# from sklearn.datasets import load_breast_cancer
# breast_cancer = load_breast_cancer()
# X = breast_cancer.data
# z= breast_cancer.target
# y = z.reshape(569,1)
# # np.random.seed(2)
# # z = np.random.randint(2, size=569)
# # y = z.reshape(569,1)


# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)

###################################################################################

#Reading dataset
np.random.seed(2)
data = pd.read_csv('diabetes.csv')

# Separation of Y and X from Dataset
Y = np.array(data['Outcome'])
y = Y.reshape(768, 1)
del data['Outcome']
X = np.array(data)
np.random.seed(2)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)


###################################################################################


# sizex = 6
# sizey = 3

# rs12 = np.random.RandomState(11)
# XA = np.round(rs12.uniform(-1, 1, (sizex, sizey)), 7)
# XA1, XA2 = share(XA, 4)

# rs13 = np.random.RandomState(22)
# YA = rs13.randint(0, 2, (sizex, 1))
# YA1, YA2 = sharey(YA)

# rs14 = np.random.RandomState(33)
# XB = np.round(rs14.uniform(-1, 1, (sizex, sizey)), 7)
# XB1, XB2 = share(XB, 6)

# rs15 = np.random.RandomState(44)
# YB = rs15.randint(0, 2, (sizex, 1))
# YB1, YB2 = sharey(YB)

###################################################################################

def secondsToStr(elapsed=None):
    if elapsed is None:
        return strftime("%Y-%m-%d %H:%M:%S", localtime())
    else:
        return str(timedelta(seconds=elapsed))

def log(s, elapsed=None):
    line = "="*40
    print(line)
    print(secondsToStr(), '-', s)
    if elapsed:
        print("Elapsed time:", elapsed)
    print(line)
    print()

def endlog():
    end = time()
    elapsed = end-start
    log("End Program", secondsToStr(elapsed))
    return elapsed

def newton_step(curr, X):
    
#     print(np.max(X.dot(curr[:, 0])))
#     print(np.min(X.dot(curr[:, 0])))
                  
#     print((X.dot(curr[:, 0])).T)
#     print((X.dot(curr[:, 0])))

   
    p = np.array(sigmoid_approximate_poly_7(X.dot(curr[:, 0])), ndmin=2).T
    
#     print(p)
    
    W = np.diag((p * (1 - p))[:, 0])

    hessian = X.T.dot(W).dot(X)
    
    
    grad = X.T.dot(y_train - p)

    
    step = np.dot(np.linalg.pinv(hessian), grad)
  
#     print(step)


    beta = curr + step

    return beta

def check_coefs_convergence(beta_old, beta_new, tol, iters):

    coef_change = np.abs(beta_old - beta_new)
    
    print('\n coef_change: ' , np.max(coef_change), '\n')

    return not (np.any(coef_change > tol) & (iters < max_iter))

start = time()
atexit.register(endlog)
log("Start Program")

beta_old, beta = np.zeros((np.size(X_train, 1), 1)), np.zeros((np.size(X_train, 1), 1))
# beta_old, beta = beta_old, beta +   1
iter_count = 0

coefs_converged = False

while not coefs_converged:

    beta_old = beta

    beta = newton_step(beta, X_train)
    
    iter_count += 1

    coefs_converged = check_coefs_convergence(beta_old, beta, tol, iter_count)

    
print('Iterations : {}'.format(iter_count))
print('Beta : \n\n{}'.format(beta))
endlog()

###################################################################################

from sklearn.metrics import accuracy_score
predict = sigmoid(X_test.dot(beta))
prediction = ((predict) > 0.5)
y_predicted = prediction.astype(int)

print(y_test.T, '\n')

print(y_predicted.T, '\n')

print('accuracy_score: ', accuracy_score(y_test, y_predicted))



In [ ]:
import sys
import warnings
import numpy as np
import time
import pandas as pd
import atexit
from time import time, strftime, localtime
from datetime import timedelta
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


np.set_printoptions(suppress=True)
np.warnings.filterwarnings('ignore')
np.set_printoptions(threshold=sys.maxsize)


def test():

    def sigmoid(x):

        return 1 / (1 + np.exp(-x))

    def sigmoid_approximate_poly_7(x):

        return 0.5 + 1.73496 * (x/8) - 4.19407 * (x/8)**3 + 5.43402 * (x/8)** 5 - 2.50739 *(x/8) ** 7

    def sigmoid_approximate_poly_5(x):

        return  0.5 + (1.53048 * (x/8)) - (2.3533056 * ((x/8) ** 3)) + (1.3511295 * ((x/8) ** 5))


    def sigmoid_approximate_poly_3(x):

        return 0.5 + 1.20096 * (x/8) - 0.81562 * (x/8)**3

    def sigmoid_approximate(x):

        return (1 / 2) + (x / 4) - (x**3 / 48) + (x**5 / 480)


    def share(input):
        np.random.seed(2)
        x = np.random.random((np.shape(input)))
        y = 0.1 *np.random.random((np.shape(input)))
        z = input - x - y

        return x , y , z

    tol=1e-6

    max_iter = 100


    #X_train1 , X_train2 = share(X_train, 1)
    #y_train1 , y_train2 = sharey(y_train)






#     #Reading dataset
#     data = pd.read_csv('pcs.csv')

#     #Separation of Y and X from Dataset
#     np.random.seed(1)

#     Y1 = data.iloc[:,-1:]
#     y = Y1.values.reshape(np.shape(data)[0],1)
#     data.drop(data.columns[len(data.columns)-1], axis=1, inplace=True)
#     X = np.array(data)


#     X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state =0)

#     #NM = X_train.T.dot(X_train)
#     #print(-4 * np.linalg.inv(NM))

#     # print(y_train)


    ###################################################################################

    # np.random.seed(1)
    # df = pd.read_csv("cancerdata.csv")
    # X = df.values[:,2:-1].astype('float64')
    # # print('\n ###################################### \n',X)

    # X = (X - np.mean(X, axis =0)) /  np.std(X, axis = 0)
    # # print('\n ###################################### \n',X)

    # ## Add a bias column to the data
    # X = np.hstack([np.ones((X.shape[0], 1)),X])
    # # print('\n ###################################### \n',X)

    # X = MinMaxScaler().fit_transform(X)
    # # print('\n ###################################### \n',X)

    # np.random.seed(1)
    # Y = df["diagnosis"].map({'M':1,'B':0})
    # Y = np.array(Y)

    # X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.25)

    ###################################################################################

    # from sklearn.datasets import load_breast_cancer
    # breast_cancer = load_breast_cancer()
    # X = breast_cancer.data
    # z= breast_cancer.target
    # y = z.reshape(569,1)
    # # np.random.seed(2)
    # # z = np.random.randint(2, size=569)
    # # y = z.reshape(569,1)


    # X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)

    ###################################################################################

    # #Reading dataset
    # np.random.seed(2)
    # data = pd.read_csv('nhanes3.csv')

    # print(data.shape)

    # # Separation of Y and X from Dataset
    # Y = np.array(data['Outcome'])
    # y = Y.reshape(768, 1)
    # del data['Outcome']
    # X = np.array(data)
    # np.random.seed(2)

    # X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state =0)

    # print(X_train)
    # print(y_train)


    ###################################################################################


    sizex = 800
    sizey = 100

    rs12 = np.random.RandomState(1)
    XA = np.round(rs12.uniform(-1, 1, (sizex, sizey)), 7)
    # XA1, XA2 = share(XA, 4)

    rs13 = np.random.RandomState(2)
    YA = rs13.randint(0, 2, (sizex, 1))
    # YA1, YA2 = sharey(YA)

    rs14 = np.random.RandomState(3)
    XB = np.round(rs14.uniform(-1, 1, (sizex, sizey)), 7)
    # XB1, XB2 = share(XB, 6)

    rs15 = np.random.RandomState(4)
    YB = rs15.randint(0, 2, (sizex, 1))
    # YB1, YB2 = sharey(YB)

    X_train = np.concatenate((XA, XB), axis=0)
    y_train = np.concatenate((YA, YB), axis=0)

    ###################################################################################

    def secondsToStr(elapsed=None):
        if elapsed is None:
            return strftime("%Y-%m-%d %H:%M:%S", localtime())
        else:
            return str(timedelta(seconds=elapsed))

    def log(s, elapsed=None):
        line = "="*40
        print(line)
        print(secondsToStr(), '-', s)
        if elapsed:
            print("Elapsed time:", elapsed)
        print(line)
        print()

    def endlog():
        end = time()
        elapsed = end-start
        log("End Program", secondsToStr(elapsed))
        return elapsed


    def newton_step(curr, X):

        p = np.array(sigmoid(X.dot(curr[:, 0])), ndmin=2).T

    #     print(p)

        W = np.diag((p * (1 - p))[:, 0])

        hessian = X.T.dot(W).dot(X)


        grad = X.T.dot(y_train - p)

    #     print(hessian)

        step = np.dot(np.linalg.pinv(hessian), grad)


        beta = curr + step

        return beta

    def check_coefs_convergence(beta_old, beta_new, tol, iters):

        coef_change = np.abs(beta_old - beta_new)

    #     print('\n coef_change: ' , np.max(coef_change), '\n')

        return not (np.any(coef_change > tol) & (iters < max_iter))


    beta_old, beta = np.zeros((np.size(X_train, 1), 1)), np.zeros((np.size(X_train, 1), 1))
    # beta_old, beta = beta_old, beta +   1
    iter_count = 0

    coefs_converged = False

    while not coefs_converged:

        beta_old = beta

        beta = newton_step(beta, X_train)

        iter_count += 1

        coefs_converged = check_coefs_convergence(beta_old, beta, tol, iter_count)
        
    return beta

# @profile
def mem1():

    beta1 = test()   
    
    return beta1

from memory_profiler import memory_usage


mem = max(memory_usage(proc=mem1))

print("Maximum memory used: {} MiB".format(mem))


In [ ]:
import sys
import warnings
import numpy as np
import time
import pandas as pd
import atexit
import queue
import concurrent.futures
from time import time, strftime, localtime
from datetime import timedelta
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

np.set_printoptions(suppress=True)
np.warnings.filterwarnings('ignore')
np.set_printoptions(threshold=sys.maxsize)
from matplotlib.pyplot import figure




def fullprint(*args, **kwargs):
    from pprint import pprint
    import numpy
    opt = numpy.get_printoptions()
    numpy.set_printoptions(threshold=numpy.inf)
    pprint(*args, **kwargs)
    numpy.set_printoptions(**opt)


##############################################################################

# def trustedـinitializer():


#     def share(input):

#         np.random.seed(1)
#         x = np.random.random((np.shape(input)))  
#         y = input - x

#     return x, y

#     rs0 = np.random.RandomState(12)
#     U = np.round(rs0.uniform(-1, 1, (1, 10)), 5)    

#     rs1 = np.random.RandomState(13)
#     V = np.round(rs1.uniform(-1, 1, (10, 1)), 5) 

#     UV = U.dot(V)

#     #################

#     U1 , U2  = share(U)

#     V1 , V2 = share(V)

#     UV1 , UV2 = share(UV)

#     return #??????????




def PPLRB(i,j):
    
    ###################################################################################
    def share(input, seed):
        np.random.seed(seed)
        x = 0.1 * np.random.random((np.shape(input)))
        y = input - x
        return x, y


    def sharey(input):
        rs4 = np.random.RandomState(24)
        a = rs4.randint(0, 2, np.shape(input))

        b = input - a

        return a, b


    ###################################################################################

    sizex = i
    sizey = j

    rs12 = np.random.RandomState(1)
    XA = np.round(rs12.uniform(-1, 1, (sizex, sizey)), 7)
    XA1, XA2 = share(XA, 4)

    rs13 = np.random.RandomState(2)
    YA = rs13.randint(0, 2, (sizex, 1))
    YA1, YA2 = sharey(YA)


    rs14 = np.random.RandomState(3)
    XB = np.round(rs14.uniform(-1, 1, (sizex, sizey)), 7)
    XB1, XB2 = share(XB, 6)

    rs15 = np.random.RandomState(4)
    YB = rs15.randint(0, 2, (sizex, 1))
    YB1, YB2 = sharey(YB)


    X_train = np.concatenate((XA, XB), axis=0)
    y_train = np.concatenate((YA, YB), axis=0)

    X_train1, X_train2 = share(X_train, 1)
    y_train1, y_train2 = sharey(y_train)

    
    
    
    

    # 
    # #Reading dataset
    # np.random.seed(2)
    # data = pd.read_csv('CancerData.csv')
    # 
    # # Separation of Y and X from Dataset
    # Y = np.array(data['Outcome'])
    # y = Y.reshape(768, 1)
    # del data['Outcome']
    # X = np.array(data)
    # np.random.seed(2)
    # 


#     data = pd.read_csv('Edinburgh.csv')

#     # Separation of Y and X from Dataset
#     np.random.seed(1)
#     Y1 = data.iloc[:, -1:]
#     y = Y1.values.reshape(np.shape(data)[0], 1)
#     data.drop(data.columns[len(data.columns) - 1], axis=1, inplace=True)
#     X = np.array(data)

#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

#     X_train1, X_train2 = share(X_train, 1)
#     y_train1, y_train2 = sharey(y_train)

    # print(X_train)
    # print(y_train)


    # NM = X_train.T.dot(X_train)
    # # print(-4 * np.linalg.inv(NM))


    ##############################################

    x = np.size(X_train, 0)
    y = np.size(X_train, 1)

#     x = sizex
#     y = sizey

    rs0 = np.random.RandomState(12)
    U = np.round(rs0.uniform(-1, 1, (y, x)), 5)

    rs1 = np.random.RandomState(12)
    V = np.round(rs1.uniform(-1, 1, (x, y)), 5)

    W = U.dot(V)

    U1, U2 = share(U, 4)

    V1, V2 = share(V, 5)

    W1, W2 = share(W, 6)

    #############################################

    rs0 = np.random.RandomState(12)
    u = np.round(rs0.uniform(-1, 1, (y, y)), 5)

    rs1 = np.random.RandomState(13)
    v = np.round(rs1.uniform(-1, 1, (y, y)), 5)

    w = u.dot(v)

    u1, u2 = share(u, 3)

    v1, v2 = share(v, 4)

    w1, w2 = share(w, 5)

    ##############################################

    rs0 = np.random.RandomState(12)
    uu = np.round(rs0.uniform(-1, 1, (x, y)), 5)

    rs1 = np.random.RandomState(13)
    vv = np.round(rs1.uniform(-1, 1, (y, 1)), 5)

    ww = uu.dot(vv)

    uu1, uu2 = share(uu, 3)

    vv1, vv2 = share(vv, 4)

    ww1, ww2 = share(ww, 5)

    ##############################################

    rs0 = np.random.RandomState(12)
    uuu = np.round(rs0.uniform(-1, 1, (x, 1)), 5)

    rs1 = np.random.RandomState(13)
    vvv = np.round(rs1.uniform(-1, 1, (x, 1)), 5)

    www = uuu * vvv

    uuu1, uuu2 = share(uuu, 3)

    vvv1, vvv2 = share(vvv, 4)

    www1, www2 = share(www, 5)

    ##############################################

    rs0 = np.random.RandomState(12)
    uuuu = np.round(rs0.uniform(-1, 1, (y, x)), 5)

    rs1 = np.random.RandomState(13)
    vvvv = np.round(rs1.uniform(-1, 1, (x, 1)), 5)

    wwww = uuuu.dot(vvvv)

    uuuu1, uuuu2 = share(uuuu, 3)

    vvvv1, vvvv2 = share(vvvv, 4)

    wwww1, wwww2 = share(wwww, 5)

    ##############################################

    rs0 = np.random.RandomState(12)
    uuuuu = np.round(rs0.uniform(-1, 1, (y, y)), 5)

    rs1 = np.random.RandomState(13)
    vvvvv = np.round(rs1.uniform(-1, 1, (y, 1)), 5)

    wwwww = uuuuu.dot(vvvvv)

    uuuuu1, uuuuu2 = share(uuuuu, 3)

    vvvvv1, vvvvv2 = share(vvvvv, 4)

    wwwww1, wwwww2 = share(wwwww, 5)


    # 1111111111111111111111111111111##############################################

    def CP1(beta):
        def mul_matrix_full(first, second, U, V, W):

            D1 = first - U
            E1 = second - V

            q1.put(D1)
            q1.put(E1)

            D2 = q2.get()
            E2 = q2.get()

            D = D1 + D2
            E = E1 + E2

            Z = W + D.dot(V) + U.dot(E) + D.dot(E)

            return Z

        def mul_number(first, second, U, V, W):

            D1 = first - U
            E1 = second - V

            q1.put(D1)
            q1.put(E1)

            D2 = q2.get()
            E2 = q2.get()

            D = D1 + D2
            E = E1 + E2

            Z = W + D * V + U * E + D * E

            return Z

        X = X_train1
        y = y_train1

        # X = np.concatenate((XA1, XB1), axis=0)
        # y = np.concatenate((YA1, YB1), axis=0)

        R = mul_matrix_full(X.T, X, U1, V1, W1)

        c = 0.00000009
        q = np.size(R, 0)
        I = np.identity(q)
        Xs = c * I
        Ms = c * R

        for x in range(0, 22):
            Ms_Xs = mul_matrix_full(Xs, Ms, u1, v1, w1)
            mul_2 = 2 * Xs
            X_s_t = mul_2 - Ms_Xs

            Ms_Ms = mul_matrix_full(Ms, Ms, u1, v1, w1)
            Ms_2 = 2 * Ms
            M_s_t = Ms_2 - Ms_Ms

            Xs = X_s_t
            Ms = M_s_t

        Hessian_Invers_TEMP = X_s_t

        Hessian_Invers = -4 * Hessian_Invers_TEMP

        for i in range(0, 10):

            beta_old = beta

            XB = mul_matrix_full(X, beta_old, uu1, vv1, ww1) * 1 / 8

            ###################################################
            Ms1 = XB

            Ms2 = mul_number(XB, XB, uuu1, vvv1, www1)
            Ms3 = mul_number(Ms2, XB, uuu1, vvv1, www1)
            Ms4 = Ms3

            Ms5 = mul_number(XB, XB, uuu1, vvv1, www1)
            Ms6 = mul_number(Ms5, XB, uuu1, vvv1, www1)
            Ms7 = mul_number(Ms6, XB, uuu1, vvv1, www1)
            Ms8 = mul_number(Ms7, XB, uuu1, vvv1, www1)
            Ms9 = Ms8

            Sig = (1.53048 * Ms1) - (2.3533056 * Ms4) + (1.3511295 * Ms9)

            #####################################################

    #         Ms1 = XB

    #         Ms2 = mul_number(XB, XB, uuu1, vvv1, www1)
    #         Ms3 = mul_number(Ms2, XB, uuu1, vvv1, www1)

    #         Ms4 = Ms3

    #         Sig = 1/2 + (1.20096 * Ms1) - (0.81562 * Ms4)


            #####################################################

    #         Ms1 = XB

    #         Ms2 = mul_number(XB, XB, uuu1, vvv1, www1)
    #         Ms3 = mul_number(Ms2, XB, uuu1, vvv1, www1)

    #         Ms4 = Ms3

    #         Ms5 = mul_number(XB, XB, uuu1, vvv1, www1)
    #         Ms6 = mul_number(Ms5, XB, uuu1, vvv1, www1)
    #         Ms7 = mul_number(Ms6, XB, uuu1, vvv1, www1)
    #         Ms8 = mul_number(Ms7, XB, uuu1, vvv1, www1)

    #         Ms9 = Ms8

    #         Ms10 = mul_number(XB, XB, uuu1, vvv1, www1)
    #         Ms11 = mul_number(Ms10, XB, uuu1, vvv1, www1)
    #         Ms12 = mul_number(Ms11, XB, uuu1, vvv1, www1)
    #         Ms13 = mul_number(Ms12, XB, uuu1, vvv1, www1)
    #         Ms14 = mul_number(Ms13, XB, uuu1, vvv1, www1)
    #         Ms15 = mul_number(Ms14, XB, uuu1, vvv1, www1)

    #         Ms16 = Ms15

    #         Sig = (1.73496 * Ms1) - (4.19407 * Ms4) + (5.43402 * Ms9) - (2.50739 * Ms16)

            #####################################################

            grad_temp = y - Sig

            Gradient = mul_matrix_full(X.T, grad_temp, uuuu1, vvvv1, wwww1)

            step = mul_matrix_full(Hessian_Invers, Gradient, uuuuu1, vvvvv1, wwwww1)

            beta = beta_old - step

        return beta


    # 2222222222222222222222222222222#####################################################
    def CP2(beta):
        def mul_matrix_full(first, second, U, V, W):

            D2 = first - U
            E2 = second - V

            q2.put(D2)
            q2.put(E2)

            D1 = q1.get()
            E1 = q1.get()

            D = D1 + D2
            E = E1 + E2

            Z = W + D.dot(V) + U.dot(E)

            return Z

        def mul_number(first, second, U, V, W):

            D2 = first - U
            E2 = second - V

            q2.put(D2)
            q2.put(E2)

            D1 = q1.get()
            E1 = q1.get()

            D = D1 + D2
            E = E1 + E2

            Z = W + D * V + U * E

            return Z

        X = X_train2
        y = y_train2

        # X = np.concatenate((XA2, XB2), axis=0)
        # y = np.concatenate((YA2, YB2), axis=0)

        R = mul_matrix_full(X.T, X, U2, V2, W2)

        c = 0.00000009
        q = np.size(R, 0)
        I = np.identity(q)
        Xs = c * I
        Ms = c * R

        for x in range(0, 22):
            Ms_Xs = mul_matrix_full(Xs, Ms, u2, v2, w2)
            mul_2 = 2 * Xs
            X_s_t = mul_2 - Ms_Xs
            Ms_Ms = mul_matrix_full(Ms, Ms, u2, v2, w2)
            Ms_2 = 2 * Ms
            M_s_t = Ms_2 - Ms_Ms

            Xs = X_s_t
            Ms = M_s_t

        Hessian_Invers_TEMP = X_s_t

        Hessian_Invers = -4 * Hessian_Invers_TEMP

        for i in range(0, 10):

            beta_old = beta

            XB = mul_matrix_full(X, beta_old, uu2, vv2, ww2) * 1 / 8

            #####################################################

            Ms1 = XB

            Ms2 = mul_number(XB, XB, uuu2, vvv2, www2)
            Ms3 = mul_number(Ms2, XB, uuu2, vvv2, www2)

            Ms4 = Ms3

            Ms5 = mul_number(XB, XB, uuu2, vvv2, www2)
            Ms6 = mul_number(Ms5, XB, uuu2, vvv2, www2)
            Ms7 = mul_number(Ms6, XB, uuu2, vvv2, www2)
            Ms8 = mul_number(Ms7, XB, uuu2, vvv2, www2)

            Ms9 = Ms8

            Sig = 1/2 + (1.53048 * Ms1) - (2.3533056 * Ms4) + (1.3511295 * Ms9)

            #####################################################

    #         Ms1 = XB

    #         Ms2 = mul_number(XB, XB, uuu2, vvv2, www2)
    #         Ms3 = mul_number(Ms2, XB, uuu2, vvv2, www2)
    #         Ms4 =  Ms3

    #         Sig = (1.20096 * Ms1) -  (0.81562 * Ms4)

            #####################################################

    #         Ms1 = XB

    #         Ms2 = mul_number(XB, XB, uuu2, vvv2, www2)
    #         Ms3 = mul_number(Ms2, XB, uuu2, vvv2, www2)

    #         Ms4 = Ms3

    #         Ms5 = mul_number(XB, XB, uuu2, vvv2, www2)
    #         Ms6 = mul_number(Ms5, XB, uuu2, vvv2, www2)
    #         Ms7 = mul_number(Ms6, XB, uuu2, vvv2, www2)
    #         Ms8 = mul_number(Ms7, XB, uuu2, vvv2, www2)

    #         Ms9 = Ms8

    #         Ms10 = mul_number(XB, XB, uuu2, vvv2, www2)
    #         Ms11 = mul_number(Ms10, XB, uuu2, vvv2, www2)
    #         Ms12 = mul_number(Ms11, XB, uuu2, vvv2, www2)
    #         Ms13 = mul_number(Ms12, XB, uuu2, vvv2, www2)
    #         Ms14 = mul_number(Ms13, XB, uuu2, vvv2, www2)
    #         Ms15 = mul_number(Ms14, XB, uuu2, vvv2, www2)

    #         Ms16 = Ms15

    #         Sig = 1 / 2 + (1.73496 * Ms1) - (4.19407 * Ms4) + (5.43402 * Ms9) - (2.50739 * Ms16)

            #####################################################

            grad_temp = y - Sig

            Gradient = mul_matrix_full(X.T, grad_temp, uuuu2, vvvv2, wwww2)

            step = mul_matrix_full(Hessian_Invers, Gradient, uuuuu2, vvvvv2, wwwww2)

            beta = beta_old - step

        return beta




    ###################################################################################

    q1 = queue.Queue()
    q2 = queue.Queue()

    import atexit
    from time import time, strftime, localtime
    from datetime import timedelta


    def secondsToStr(elapsed=None):
        if elapsed is None:
            return strftime("%Y-%m-%d %H:%M:%S", localtime())
        else:
            return str(timedelta(seconds=elapsed))


    def log(s, elapsed=None):
        line = "=" * 40
        print()
        print(line)
        print(secondsToStr(), '-', s)
        if elapsed:
            print("Elapsed time:", elapsed)
        print(line)
        print()


    def endlog():
        end = time()
        elapsed = end - start
        log("End Program", secondsToStr(elapsed))


    def SPDZ():
        
        beta_old, beta = np.zeros((np.size(X_train, 1), 1)), np.zeros((np.size(X_train, 1), 1))

        # beta = np.zeros((np.size(XA+XB, 1), 1))

        beta1, beta2 = share(beta, 21)

        with concurrent.futures.ThreadPoolExecutor() as executor:
            f1 = executor.submit(CP1, beta1)
            f2 = executor.submit(CP2, beta2)

            result = (f1.result() + f2.result())

            #         print('\n', f1.result(), '\n \n', f2.result(), '\n \n')

            #         print('result\n' , np.max(result))

            return result

        return ret
        
    start = time()
    beta = SPDZ()
    end = time()
    elapsed = end - start
    print('Elapsed :', end - start)
    return beta


print(PPLRB(800,100))

# def mem1():

#     beta1 = PPLRB()  
    
#     print(beta1)
    
    
#     return beta1


# from memory_profiler import memory_usage


# mem = max(memory_usage(proc=mem1))

# print("Maximum memory used: {} MiB".format(mem))



In [ ]:
import sys
import warnings
import numpy as np
import time
import pandas as pd
import atexit
from time import time, strftime, localtime
from datetime import timedelta
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler



np.set_printoptions(suppress=True)
np.warnings.filterwarnings('ignore')
np.set_printoptions(threshold=sys.maxsize)



def sigmoid(x):

    return 1 / (1 + np.exp(-x))

def sigmoid_approximate_poly_7(x):
    
    return 0.5 + 1.73496 * (x/8) - 4.19407 * (x/8)**3 + 5.43402 * (x/8)** 5 - 2.50739 *(x/8) ** 7

def sigmoid_approximate_poly_5(x):
    
    return  0.5 + (1.53048 * (x/8)) - (2.3533056 * ((x/8) ** 3)) + (1.3511295 * ((x/8) ** 5))


def sigmoid_approximate_poly_3(x):

    return 0.5 + 1.20096 * (x/8) - 0.81562 * (x/8)**3

def sigmoid_approximate(x):

    return (1 / 2) + (x / 4) - (x**3 / 48) + (x**5 / 480)


def share(input):
    np.random.seed(2)
    x = np.random.random((np.shape(input)))
    y = 0.1 *np.random.random((np.shape(input)))
    z = input - x - y

    return x , y , z

tol=1e-6

max_iter = 100


#X_train1 , X_train2 = share(X_train, 1)
#y_train1 , y_train2 = sharey(y_train)






#Reading dataset
data = pd.read_csv('uis.csv')

#Separation of Y and X from Dataset 
np.random.seed(1)

Y1 = data.iloc[:,-1:]
y = Y1.values.reshape(np.shape(data)[0],1)
data.drop(data.columns[len(data.columns)-1], axis=1, inplace=True)
X = np.array(data)


x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state =0)

from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression()
logisticRegr.fit(x_train, y_train)

logisticRegr.predict(x_test)
predictions = logisticRegr.predict(x_test)

score = logisticRegr.score(x_test, y_test)
print(score)